<img src="https://raw.githubusercontent.com/reactive-python/reactpy/main/branding/svg/reactpy-logo-landscape.svg" alt="ReactPy Logo" style="min-width: 300px; width: 35%" />

---

[ReactPy](https://reactpy.dev/) is a library for building user interfaces in Python without Javascript. ReactPy interfaces are made from components which look and behave similarly to those found in [ReactJS](https://reactjs.org/). Designed with simplicity in mind, ReactPy can be used by those without web development experience while also being powerful enough to grow with your ambitions.

# At a Glance

To get a rough idea of how to write apps in ReactPy, take a look at the tiny [“hello world”](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program) application below:

In [1]:
from reactpy import component, html


@component
def App():
    return html.h1("Hello, World!")


App()

LayoutWidget(Layout(ContextProvider(<function context at 0x7f7986b10cc0>)))

# Creating Interfaces

ReactPy is a Python package for making user interfaces (UI). These interfaces are built from small elements of functionality like buttons text and images. ReactPy allows you to combine these elements into reusable “components”. Once you learn how these UI elements are created and organized into components you'll be able to do things like create interfaces from raw data:

<a href="https://reactpy.dev/docs/guides/creating-interfaces/index.html" target="_blank"><button>Read More!</button></a>

In [2]:
from reactpy import component, html


@component
def DataList(items, filter_by_priority=None, sort_by_priority=False):
    if filter_by_priority is not None:
        items = [i for i in items if i["priority"] <= filter_by_priority]
    if sort_by_priority:
        items = list(sorted(items, key=lambda i: i["priority"]))
    list_item_elements = [html.li({"key": i["id"]}, i["text"]) for i in items]
    return html.ul(list_item_elements)


@component
def TodoList():
    tasks = [
        {"id": 0, "text": "Make breakfast", "priority": 0},
        {"id": 1, "text": "Feed the dog", "priority": 0},
        {"id": 2, "text": "Do laundry", "priority": 2},
        {"id": 3, "text": "Go on a run", "priority": 1},
        {"id": 4, "text": "Clean the house", "priority": 2},
        {"id": 5, "text": "Go to the grocery store", "priority": 2},
        {"id": 6, "text": "Do some coding", "priority": 1},
        {"id": 7, "text": "Read a book", "priority": 1},
    ]
    return html.section(
        html.h1("My Todo List"),
        DataList(tasks, filter_by_priority=1, sort_by_priority=True),
    )


TodoList()

LayoutWidget(Layout(ContextProvider(<function context at 0x7f7986b10cc0>)))

# Adding Interactivity

Components often need to change what’s on the screen as a result of an interaction. For example, typing into the form should update the input field, and clicking a “Comment” button should bring up a text input field, clicking “Buy” should put a product in the shopping cart. Components need to “remember” things like the current input value, the current image, and the shopping cart. In ReactPy, this kind of component-specific memory is created and updated with a “hook” called use_state() that creates a state variable and state setter respectively:

<a href="https://reactpy.dev/docs/guides/adding-interactivity/index.html" target="_blank"><button>Read More!</button></a>

In [3]:
import json
from pathlib import Path

from reactpy import component, use_state, html


DATA_PATH = Path().parent / "data" / "gallery-data.json"
sculpture_data = json.loads(DATA_PATH.read_text())


@component
def Gallery():
    index, set_index = use_state(0)

    def handle_click(event):
        set_index(index + 1)

    bounded_index = index % len(sculpture_data)
    sculpture = sculpture_data[bounded_index]
    alt = sculpture["alt"]
    artist = sculpture["artist"]
    description = sculpture["description"]
    name = sculpture["name"]
    url = sculpture["url"]

    return html.div(
        html.button({"onClick": handle_click}, "Next"),
        html.h2(name, " by ", artist),
        html.p(f"({bounded_index + 1} or {len(sculpture_data)})"),
        html.img({"src": url, "alt": alt, "style": {"height": "200px"}}),
        html.p(description),
    )


Gallery()

LayoutWidget(Layout(ContextProvider(<function context at 0x7f7986b10cc0>)))

# Using ReactPy With Jupyter Widgets

It's possible to use Jupyter Widgets in ReactPy components if you convert them first using `reactpy_jupyter.from_widget`.

In [4]:
from reactpy_jupyter import from_widget
from ipywidgets import IntSlider

slider_widget = IntSlider()
slider_component = from_widget(slider_widget)

slider_component

LayoutWidget(Layout(ContextProvider(<function context at 0x7f7986b10cc0>)))

Let's consider a ReactPy component that responds to and displays changes from an `ipywidgets.IntSlider`. The ReactPy component will need to accept an `IntSlider` instance as one of its arguments, convert it to a component with `from_widget`, declare state that will track the slider's value, and register a lister that will update that state via the slider's `IntSlider.observe()` method using an ["effect"](https://reactpy.dev/docs/reference/hooks-api.html#use-effect):

In [5]:
from reactpy import use_effect
from reactpy_jupyter import from_widget


@component
def SliderObserver(slider):
    slider_component = from_widget(slider)
    value, set_value = use_state(0)

    @use_effect
    def register_observer():
        def handle_change(change):
            set_value(change["new"])

        # observe the slider's value
        slider.observe(handle_change, "value")
        # unobserve the slider's value if this component is no longer displayed
        return lambda: slider.unobserve(handle_change, "value")

    return html.div(
        slider_component, html.p(f"ReactPy observes the value to be: ", value)
    )

Now you need to pass the `SliderObserver` component an `IntSlider` widget and display it.


In [6]:
from ipywidgets import IntSlider

SliderObserver(IntSlider(readout=False))

LayoutWidget(Layout(ContextProvider(<function context at 0x7f7986b10cc0>)))

You can also include ReactPy components within Jupyter Widgets using `reactpy_jupyter.to_widget`

In [8]:
from ipywidgets import Box
from reactpy_jupyter import to_widget

slider = IntSlider(readout=False)
slider_observer_widget = to_widget(SliderObserver(slider))

Box([slider, slider_observer_widget])

Box(children=(IntSlider(value=0, readout=False), LayoutWidget(Layout(ContextProvider(<function context at 0x7f…

If it becomes painful to convert every ReactPy component to a jupyter widget you can create an alternate widget constructor:

In [9]:
slider = IntSlider(readout=False)
slider_observer_constructor = to_widget(SliderObserver)
observer_1 = slider_observer_constructor(slider)
observer_2 = slider_observer_constructor(slider)

Box([observer_1, observer_2])

Box(children=(LayoutWidget(Layout(ContextProvider(<function context at 0x7f7986b10cc0>))), LayoutWidget(Layout…

# Javascript Integration

While ReactPy is a great tool for displaying HTML and responding to browser events with pure Python, there are other projects which already allow you to do this inside Jupyter Notebooks or in standard web apps. The real power of ReactPy comes from its ability to seamlessly leverage the existing Javascript ecosystem:

<a href="https://reactpy.dev/docs/guides/escape-hatches/javascript-components.html" target="_blank"><button>Read More!</button></a>

In [ ]:
from reactpy import component, web


victory = web.module_from_template("react", "victory-bar", fallback="⌛")
VictoryBar = web.export(victory, "VictoryBar")


@component
def Demo():
    bar_style = {"parent": {"width": "500px"}, "data": {"fill": "royalblue"}}
    return VictoryBar({"style": bar_style})


Demo()

# [Learn More!](https://reactpy.dev/docs/index.html)